# سفری سفارشات کے لیے متوازی ترتیب کاری

یہ نوٹ بُک مائیکروسافٹ ایجنٹ فریم ورک کا استعمال کرتے ہوئے **متوازی ترتیب کاری** کو ظاہر کرتی ہے۔ ہم ایک سفری سفارشات کا نظام بنائیں گے جس میں تین مخصوص ایجنس شامل ہوں گے جو متوازی طور پر کام کرتے ہیں تاکہ جامع سفری معلومات فراہم کی جا سکیں۔

## آپ کیا سیکھیں گے:
1. **متوازی ترتیب کاری**: متعدد ایجنٹوں کا متوازی چلنا (فین آؤٹ/فین ان پیٹرن)
2. **ConcurrentBuilder**: متوازی ورک فلو بنانے کے لیے اعلی سطح کا API
3. **سفری سفارشات**: تین مخصوص ایجنس مل کر کام کرتے ہیں
4. **ڈیفالٹ اجتماع**: متعدد ایجنٹوں کے جوابات کو یکجا کرنا
5. **کارکردگی کے فوائد**: متوازی اجرا بمقابلہ متسلسل عملدرآمد

## تین مخصوص ایجنس:

1. **Attractions Agent**: سیاحتی مقامات، سرگرمیاں، مشہور مقامات
2. **Dining Agent**: مقامی کھانے، ریستوران، خوراک کے تجربات
3. **History Agent**: تاریخی حقائق، ثقافتی اہمیت، سیاق و سباق


In [3]:
import asyncio
import json
import os
from typing import Any, cast

from agent_framework import (
    ChatMessage,
    ConcurrentBuilder,
    WorkflowOutputEvent,
)

# GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("All imports successful!")

All imports successful!


## مرحلہ 1: منظم نتائج کے لیے پائیڈینٹک ماڈلز کی تعریف کریں

یہ ماڈلز اس خاکہ کی تعریف کرتے ہیں جو ہر مخصوص ایجنٹ واپس کرے گا۔ یہ تمام ایجنٹس سے مستحکم اور قابل تجزیہ جوابات کو یقینی بناتا ہے۔


## مرحلہ 1: منظم نتائج کے لیے Pydantic ماڈلز کی تعریف کریں

یہ ماڈلز وہ اسکیمہ بیان کرتے ہیں جو ہر مخصوص ایجنٹ واپس کرے گا۔ اس سے تمام ایجنٹوں سے یکساں اور قابلِ تجزیہ جوابات یقینی بنتے ہیں۔


In [6]:
class AttractionsRecommendation(BaseModel):
    """Tourist attractions and activities recommendations."""

    destination: str
    top_attractions: list[str]
    activities: list[str]
    best_time_to_visit: str
    transportation_tips: str  


class DiningRecommendation(BaseModel):
    """Food and dining recommendations."""

    destination: str
    local_cuisine: str
    must_try_dishes: list[str]
    recommended_restaurants: list[str]
    food_experiences: list[str]
    dining_etiquette: str


class HistoryRecommendation(BaseModel):
    """Historical and cultural information."""

    destination: str
    historical_significance: str
    cultural_highlights: list[str]
    important_periods: list[str]
    cultural_experiences: list[str]
    interesting_facts: list[str]

## Step 2: لوڈ کریں اینوائرمنٹ ویریبلز

LLM کلائنٹ (GitHub ماڈلز یا OpenAI) کو وہی پیٹرن استعمال کرتے ہوئے ترتیب دیں جیسا کہ مڈل ویئر نوٹ بک میں کیا گیا ہے۔


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id="gpt-4o"
)

## مرحلہ 3: تین خصوصی سفری ایجنٹس بنائیں


In [11]:
# Agent 1: Tourist Attractions Expert
attractions_agent = chat_client.create_agent(
    instructions=(
        "You are a tourism expert specializing in attractions and activities. "
        "When given a travel destination, provide comprehensive recommendations for "
        "tourist attractions, activities, best times to visit, and transportation tips. "
        "Focus on popular landmarks, unique experiences, and practical travel advice. "
        "Return structured JSON with the specified fields."
    ),
    name="attractions_agent",
    response_format=AttractionsRecommendation,
)

# Agent 2: Food and Dining Expert
dining_agent = chat_client.create_agent(
    instructions=(
        "You are a culinary expert specializing in local food and dining experiences. "
        "When given a travel destination, provide recommendations for local cuisine, "
        "must-try dishes, recommended restaurants, and unique food experiences. "
        "Include dining etiquette and cultural food customs. "
        "Return structured JSON with the specified fields."
    ),
    name="dining_agent",
    response_format=DiningRecommendation,
)


# Agent 3: History and Culture Expert
history_agent = chat_client.create_agent(
    instructions=(
        "You are a historian and cultural expert. "
        "When given a travel destination, provide historical context, cultural significance, "
        "important historical periods, cultural experiences, and interesting facts. "
        "Focus on helping travelers understand the cultural heritage and historical importance. "
        "Return structured JSON with the specified fields."
    ),
    name="history_agent",
    response_format=HistoryRecommendation,
)

# Step 4: ایک ساتھ متعدد ورک فلو بنائیں

The ConcurrentBuilder ایک ایسا ورک فلو بناتا ہے جو:
1. تمام تین ایجنٹس کو ایک ساتھ ایک ہی ان پٹ بھیجتا ہے (fan-out)
2. بہتر کارکردگی کے لیے ایجنٹس کو متوازی طور پر چلایا جاتا ہے
3. تمام جوابات کو ایک واحد آؤٹ پٹ میں جمع کرتا ہے (fan-in)
4. تمام ایجنٹس سے مشترکہ ChatMessage فہرست واپس کرتا ہے


In [12]:
# Build the concurrent workflow using ConcurrentBuilder
workflow = (
    ConcurrentBuilder()
    .participants([attractions_agent, dining_agent, history_agent])
    .build()
)

display(HTML("""
<div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
    <h3 style='margin: 0 0 15px 0;'>Concurrent Workflow Built Successfully!</h3>
    <p style='margin: 0; line-height: 1.6;'>
        <strong>Architecture:</strong><br>
        • Input → <strong>Dispatcher</strong> (fan-out)<br>
        • <strong>3 Agents</strong> run in parallel (attractions, dining, history)<br>
        • <strong>Aggregator</strong> combines results (fan-in)<br>
        • Output → Combined travel recommendations
    </p>
</div>
"""))

## Step 5: ٹیسٹ کیس 1 - ٹوکیو کی سفری سفارشات

آئیے ٹوکیو کو منزل کے طور پر استعمال کرتے ہوئے ہماری ہم وقت سازی والے ورک فلو کا تجربہ کریں۔ تینوں ایجنٹ بیک وقت کام کریں گے تاکہ جامع سفری سفارشات فراہم کر سکیں۔


In [1]:
async def display_travel_recommendations(destination: str):
    """Run the concurrent workflow and display formatted results."""

    display(HTML(f"""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Processing Travel Recommendations for {destination}</h3>
        <p style='margin: 0;'><strong>Status:</strong> Running 3 agents concurrently...</p>
    </div>
    """))

    # Run the workflow
    events = await workflow.run(f"I want comprehensive travel recommendations for {destination}")
    outputs = events.get_outputs()

    if outputs:
        # Get the aggregated messages from all agents
        messages: list[ChatMessage] = outputs[0]
        # Separate messages by agent (skip user message)
        agent_responses = [msg for msg in messages if msg.author_name in [
            "attractions_agent", "dining_agent", "history_agent"]]

        # Display results
        display(HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h2 style='margin: 0 0 20px 0;'>Complete Travel Guide for {destination}</h2>
            <p style='margin: 0; font-size: 14px; opacity: 0.9;'>Generated by 3 concurrent agents</p>
        </div>
        """))
        # Process each agent's response
        for msg in agent_responses:
            agent_name = msg.author_name

            try:
                # Parse the JSON response
                if agent_name == "attractions_agent":
                    data = AttractionsRecommendation.model_validate_json(
                        msg.text)
                    display_attractions_section(data)
                elif agent_name == "dining_agent":
                    data = DiningRecommendation.model_validate_json(msg.text)
                    display_dining_section(data)
                elif agent_name == "history_agent":
                    data = HistoryRecommendation.model_validate_json(msg.text)
                    display_history_section(data)
            except Exception as e:
                display(HTML(f"""
                <div style='padding: 15px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>Error parsing {agent_name} response:</strong> {str(e)}
                    <details><summary>Raw response</summary>{msg.text}</details>
                </div>
                """))
def display_attractions_section(data: AttractionsRecommendation):
    """Display attractions recommendations in a formatted section."""
    attractions_list = "".join([f"<li>{attraction}</li>" for attraction in data.top_attractions])
    activities_list = "".join([f"<li>{activity}</li>" for activity in data.activities])
    
    display(HTML(f"""
    <div style='padding: 20px; background: #e3f2fd; border-radius: 8px; margin: 15px 0; border-left: 4px solid #2196f3;'>
        <h3 style='margin: 0 0 15px 0; color: #1976d2;'>🏛️ Tourist Attractions & Activities</h3>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Top Attractions:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{attractions_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
        <h4 style='margin: 0 0 8px 0; color: #333;'>Recommended Activities:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{activities_list}</ul>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Best Time to Visit:</strong> {data.best_time_to_visit}
        </div>
        <div>
            <strong style='color: #333;'>Transportation Tips:</strong> {data.transportation_tips}
        </div>
    </div>
    """))


def display_dining_section(data: DiningRecommendation):
    """Display dining recommendations in a formatted section."""
    dishes_list = "".join(
        [f"<li>{dish}</li>" for dish in data.must_try_dishes])
    restaurants_list = "".join(
        [f"<li>{restaurant}</li>" for restaurant in data.recommended_restaurants])
    experiences_list = "".join(
        [f"<li>{exp}</li>" for exp in data.food_experiences])

    display(HTML(f"""
    <div style='padding: 20px; background: #fff3e0; border-radius: 8px; margin: 15px 0; border-left: 4px solid #ff9800;'>
        <h3 style='margin: 0 0 15px 0; color: #f57c00;'>🍜 Food & Dining Experiences</h3>
        <div style='margin-bottom: 15px;'>
            <strong style='color: #333;'>Local Cuisine:</strong> {data.local_cuisine}
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Must-Try Dishes:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{dishes_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Recommended Restaurants:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{restaurants_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Food Experiences:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{experiences_list}</ul>
        </div>
        <div>
            <strong style='color: #333;'>Dining Etiquette:</strong> {data.dining_etiquette}
        </div>
    </div>
    """))


def display_history_section(data: HistoryRecommendation):
    """Display history recommendations in a formatted section."""
    highlights_list = "".join(
        [f"<li>{highlight}</li>" for highlight in data.cultural_highlights])
    periods_list = "".join(
        [f"<li>{period}</li>" for period in data.important_periods])
    experiences_list = "".join(
        [f"<li>{exp}</li>" for exp in data.cultural_experiences])
    facts_list = "".join(
        [f"<li>{fact}</li>" for fact in data.interesting_facts])

    display(HTML(f"""
    <div style='padding: 20px; background: #f3e5f5; border-radius: 8px; margin: 15px 0; border-left: 4px solid #9c27b0;'>
        <h3 style='margin: 0 0 15px 0; color: #7b1fa2;'>📚 History & Culture</h3>
        <div style='margin-bottom: 15px;'>
            <strong style='color: #333;'>Historical Significance:</strong> {data.historical_significance}
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Cultural Highlights:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{highlights_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Important Historical Periods:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{periods_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Cultural Experiences:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{experiences_list}</ul>
        </div>
        <div>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Interesting Facts:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{facts_list}</ul>
        </div>
    </div>
    """))

    # Test with Tokyo
await display_travel_recommendations("Tokyo")

NameError: name 'AttractionsRecommendation' is not defined

# مرحلہ 6: ٹیسٹ کیس 2 - پیرس سفر کی سفارشات


In [14]:
await display_travel_recommendations("Paris")

## مرحلہ 7: کارکردگی کا تجزیہ - متوازی بمقابلہ ترتیبی

آئیں ہم متوازی اور ترتیبی عملدرآمد کے درمیان کارکردگی کے فرق کو ماپیں تاکہ متوازی آرکیسٹریشن کے فوائد کو ظاہر کیا جا سکے۔


In [15]:
import time
from agent_framework import SequentialBuilder


async def measure_concurrent_performance(destination: str):
    """Measure concurrent execution time."""
    start_time = time.time()

    events = await workflow.run(f"I want travel recommendations for {destination}")
    outputs = events.get_outputs()

    end_time = time.time()
    return end_time - start_time, len(outputs[0]) if outputs else 0


async def measure_sequential_performance(destination: str):
    """Measure sequential execution time."""
    # Build sequential workflow for comparison
    sequential_workflow = (
        SequentialBuilder()
        .participants([attractions_agent, dining_agent, history_agent])
        .build()
    )
    start_time = time.time()

    events = await sequential_workflow.run(f"I want travel recommendations for {destination}")
    outputs = events.get_outputs()

    end_time = time.time()
    return end_time - start_time, len(outputs[0]) if outputs else 0


async def performance_comparison():
    """Compare concurrent vs sequential performance."""
    test_destination = "Barcelona"

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Performance Comparison Test</h3>
        <p style='margin: 0;'>Testing with destination: <strong>Barcelona</strong></p>
    </div>
    """))

    # Test concurrent execution
    print("Running concurrent workflow...")
    concurrent_time, concurrent_msgs = await measure_concurrent_performance(test_destination)

# Test sequential execution
    print("Running sequential workflow...")
    sequential_time, sequential_msgs = await measure_sequential_performance(test_destination)

    # Calculate performance improvement
    improvement = ((sequential_time - concurrent_time) / sequential_time) * 100

    display(HTML(f"""
    <div style='padding: 25px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 12px; 
                box-shadow: 0 4px 12px rgba(102,126,234,0.4); margin: 20px 0;'>
        <h2 style='margin: 0 0 20px 0;'>Performance Results</h2>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 20px;'>
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h4 style='margin: 0 0 10px 0;'>⚡ Concurrent Execution</h4>
                <p style='margin: 0; font-size: 24px; font-weight: bold;'>{concurrent_time:.2f}s</p>
                <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>{concurrent_msgs} messages</p>
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h4 style='margin: 0 0 10px 0;'>🔄 Sequential Execution</h4>
                <p style='margin: 0; font-size: 24px; font-weight: bold;'>{sequential_time:.2f}s</p>
                <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>{sequential_msgs} messages</p>
            </div>
        </div>
        <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;'>
            <h4 style='margin: 0 0 10px 0;'>Performance Improvement</h4>
            <p style='margin: 0; font-size: 20px; font-weight: bold;'>{improvement:.1f}% faster</p>
            <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>
                Saved {sequential_time - concurrent_time:.2f} seconds with concurrent execution
            </p>
        </div>
    </div>
    """))
# Run performance comparison
await performance_comparison()

Running concurrent workflow...
Running sequential workflow...


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ڈس کلیمر**:
اس دستاویز کو AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کی مدد سے ترجمہ کیا گیا ہے۔ اگرچہ ہم درستگی کی کوشش کرتے ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا بے ضابطگیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں معتبر ماخذ سمجھا جانا چاہیے۔ اہم معلومات کے لئے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم قبول نہیں کرتے۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
